Part One

In [ ]:
import numpy as np
import galois
from itertools import product

with open("input10.txt", "r") as f:
    manual = f.readlines()

targets, all_buttons = [line.split()[0].strip('[]') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [tuple(0 if ch=="." else 1 for ch in t) for t in targets], [[tuple(map(int,v.strip("()").split(","))) for v in lst] for lst in all_buttons]
all_buttons = [[tuple(1 if i in v else 0 for i in range(len(k))) for v in lst] for k,lst in zip(targets,all_buttons)]

sum_min_presses = 0
for target, buttons in zip(targets,all_buttons):
    target, buttons = np.array(target).reshape(-1,1), np.array(buttons)
    matrix = galois.GF(2)(np.hstack((buttons.T, target)))
    RREF = matrix.row_reduce()

    #from RREF, gets a dictionary with the possible values of the independent variables and
    #and another with the dependence relations of the dependent ones
    independent_vars, dependent_vars = {}, {}
    for row in RREF[::-1]:
        if (row == 0).all(): continue
        whereone = np.argwhere(row[:-1])
        if whereone.size==1: independent_vars[int(whereone[0,0])]={int(row[-1])}; continue
        for j in whereone[1:].reshape(-1,): independent_vars[int(j)] = {0,1}
        dependent_vars[int(whereone[0,0])] = (tuple(int(i) for i in whereone[1:].reshape(-1,)),int(row[-1]))

    #first an iterable only with tuples filled with the independent vars (elsewhere placeholders)
    pre_possibles = product(*[{j for j in independent_vars[i]} if i in independent_vars else {-1} for i in range(RREF.shape[1]-1)])
    #now populate with all possible tuples
    possibles = set()
    for poss in pre_possibles:
        poss_lst = list(poss)
        for i in dependent_vars:
            poss_lst[i] = (sum(poss[j] for j in dependent_vars[i][0])+dependent_vars[i][1]) %2
        possibles.add(tuple(poss_lst))
    del pre_possibles
    sum_min_presses+= min(sum(poss) for poss in possibles)
sum_min_presses

447

Part Two

Now without galois GF(2) fields

In [218]:
import sympy as sp

with open("example_input10.txt", "r") as f:
    manual = f.readlines()

targets, all_buttons = [line.split('{')[1].strip('}\n') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual[:1]]
targets, all_buttons = [tuple(int(i) for i in t.split(',')) for t in targets], [[tuple(map(int,v.strip("()").split(","))) for v in lst] for lst in all_buttons]
all_buttons = [[tuple(1 if i in v else 0 for i in range(len(k))) for v in lst] for k,lst in zip(targets,all_buttons)]

sum_min_presses = 0
for target, buttons in zip(targets,all_buttons):
    target, buttons = sp.Matrix(target), sp.Matrix(buttons)
    matrix = sp.Matrix.hstack(buttons.T, target)
    RREF = matrix.rref()[0]
    
    #from RREF, gets a dictionary with the possible values of the independent variables and
    #and another with the dependence relations of the dependent ones
    independent_vars, dependent_vars = {}, {}
    # for row in RREF[::-1]:
    for i in range(RREF.shape[0]):
        row = RREF[i,:]
        if row == 0: continue
        wherenonzero = np.argwhere(row[:-1])#[:,1] #column 0 is trivial
        if wherenonzero.size==1: independent_vars[int(wherenonzero[0,0])]={int(row[-1])}; continue
        dependent_vars[int(wherenonzero[0,0])] = (int(row[-1]),tuple(-row[i]*i for i in wherenonzero[1:].reshape(-1,)))
        for j in wherenonzero[1:].reshape(-1,):
            if int(j) not in independent_vars: independent_vars[int(j)] = [0,np.inf]
        if wherenonzero.size==2:
            if row[wherenonzero[1,0]]>0:
                upper_bound = row[-1]
                independent_vars[wherenonzero[1,0]][1] = min(upper_bound, independent_vars[wherenonzero[1,0]][1])
            elif row[-1]<0:
                lower_bound = -row[-1]
                independent_vars[wherenonzero[1,0]][0] = max(lower_bound, independent_vars[wherenonzero[1,0]][0])
RREF

Matrix([
[1, 0, 0, 1, 0, -1, 2],
[0, 1, 0, 0, 0,  1, 5],
[0, 0, 1, 1, 0, -1, 1],
[0, 0, 0, 0, 1,  1, 3]])

In [219]:
row, row == 0, np.argwhere(row), np.argwhere(row)[:,1], dependent_vars, independent_vars

(Matrix([[0, 0, 0, 0, 1, 1, 3]]),
 False,
 array([[0, 4],
        [0, 5],
        [0, 6]]),
 array([4, 5, 6]),
 {0: (2, (-3, 5)), 1: (5, (-5,)), 2: (1, (-3, 5)), 4: (3, (-5,))},
 {3: [0, inf], 5: [0, 3]})

maybe for the inf-upper-bounded we do it iteratively until some number goes negative

In [178]:
RREF

Matrix([
[1, 0, 0,  1,  6],
[0, 1, 0, -1, -1],
[0, 0, 1,  0,  5],
[0, 0, 0,  0,  0],
[0, 0, 0,  0,  0],
[0, 0, 0,  0,  0]])

.

.

.

.

.

.

.

.

###### little test shows how galois reduce_row is really different from straight rref

In [ ]:
import sympy as sp
for target, buttons in zip(targets[1:2],all_buttons):
    target, buttons = np.array(target).reshape(-1,1), np.array(buttons)
    matrix = galois.GF(2)(np.hstack((buttons.T, target)))
    RREF = matrix.row_reduce()
    print(RREF)
    print(sp.Matrix.hstack(sp.Matrix(buttons.T), sp.Matrix(target)).rref())

[[1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 1 1]
 [0 0 0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 1 1 0]]
(Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,    0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0,  1,    0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1, -2/3],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0,  0, -1/3],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0,  1,  2/3],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0,  1,  7/3],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0,  0, -2/3],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0,  1,  1/3],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1,  1/3],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1,   -2]]), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9))
